In [10]:
!pip install kaleido

In [11]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook"
%matplotlib inline
plt.style.use('ggplot')
import kaleido

In [12]:
# Jupyter Specifics
from IPython.display import HTML
from ipywidgets.widgets import interact, IntSlider, FloatSlider, Layout

style = {'description_width': '100px'}
slider_layout = Layout(width='99%')

In [26]:

def ode_model(z, t, N, gamma, mu):
    """
    Reference https://www.idmod.org/docs/hiv/model-seir.html
    """
    G, MU, MS = z
    dGdt = N - gamma*G - mu*G
    dMUdt = gamma*G
    dMSdt = mu*G
    return [dGdt, dMUdt, dMSdt]


def ode_solver(t, initial_conditions, params):
    initG, initMU, initMS = initial_conditions
    N,  gamma, mu = params
    res = odeint(ode_model, [initG, initMU, initMS], t, args=(N, gamma, mu))
    return res

# ref: https://www.medrxiv.org/content/10.1101/2020.04.01.20049825v1.full.pdf
initG = 32 
initMU = 0
initMS = 0

sigma = 0.9
gamma = 0.08
mu = 0.008
N = 2
time = 180

def main(initG, initMU, initMS, N, gamma, mu, time):
    initial_conditions = [initG, initMU, initMS]
    params = [N, gamma, mu]
    tspan = np.arange(0, time, 1)
    sol = ode_solver(tspan, initial_conditions, params)
    G, MU, MS = sol[:, 0], sol[:, 1], sol[:, 2]
    
    # Create traces
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=tspan, y=G, mode='lines+markers', name='Golgi'))
    fig.add_trace(go.Scatter(x=tspan, y=MU, mode='lines+markers', name='Memb. Up'))
    fig.add_trace(go.Scatter(x=tspan, y=MS, mode='lines+markers', name='Memb. Side'))

    if days <= 30:
        step = 1
    elif days <= 90:
        step = 7
    else:
        step = 30
    
    # Edit the layout
    fig.update_layout(title='Simulation of Compartmental Model',
                       xaxis_title='Time',
                       yaxis_title='Counting unit',
                       title_x=0.5,
                      width=900, height=600
                     )
    fig.update_xaxes(tickangle=-90, tickformat = None, tickmode='array', tickvals=np.arange(0, days + 1, step))
    fig.show()

In [27]:

interact(main, 
         initG=IntSlider(min=0, max=100, step=1, value=initG, description='Conc. Golgi', style=style, layout=slider_layout),
         initMU=IntSlider(min=0, max=100, step=1, value=initMU, description='Conc. Membrane Up', style=style, layout=slider_layout),
         initMS=IntSlider(min=0, max=100, step=1, value=initMS, description='Conc. Membrane Side', style=style, layout=slider_layout),

         N=FloatSlider(min=0, max=4, step=0.001, value=N, description='Transfer from Nuclei', style=style, layout=slider_layout),
         gamma=FloatSlider(min=0, max=4, step=0.001, value=gamma, description='Ratio Golgi to Memb. Up', style=style, layout=slider_layout),
         mu=FloatSlider(min=0, max=1, step=0.001, value=mu, description='Ratio Golgi to Memb. Side', style=style, layout=slider_layout),
         time=IntSlider(min=1, max=600, step=1, value=days, description='Time (min)', style=style, layout=slider_layout),
        )

interactive(children=(IntSlider(value=32, description='Conc. Golgi', layout=Layout(width='99%'), style=SliderS…

<function __main__.main(initG, initMU, initMS, N, gamma, mu, time)>